In [35]:
#dependencies and libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from sklearn.preprocessing import MinMaxScaler
from numpy.random import seed
from tensorflow import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import glob
%matplotlib inline

In [48]:
root_directory = r"Resources"

all_files = glob.glob(f'{root_directory}\*.csv')

li = []

for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0,)
    frame["Year"] = filename[10:14]
    li.append(frame)

df = pd.concat(li, axis=0, ignore_index=True)

In [49]:
#creating blank dataframe for csv data
df

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt
0,0,Ron Johnson,NYG,RB,23.0,14.0,14.0,0.0,0.0,0.0,...,0.0,0.0,1027.0,8.0,263.0,487.0,4.0,261.40,1970,NaN
1,1,Dick Gordon,CHI,WR,26.0,14.0,14.0,0.0,0.0,0.0,...,0.0,0.0,17.0,0.0,4.0,1026.0,13.0,249.30,1970,NaN
2,2,Gene Washington,SFO,WR,23.0,13.0,13.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1100.0,12.0,235.00,1970,NaN
3,3,Gary Garrison,SDG,WR,26.0,14.0,14.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,4.0,1006.0,12.0,217.30,1970,NaN
4,4,MacArthur Lane,STL,RB,28.0,14.0,14.0,0.0,0.0,0.0,...,0.0,0.0,977.0,11.0,206.0,365.0,2.0,240.20,1970,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26123,615,Ray-Ray McCloud,CAR,0,23.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.00,2019,0.0
26124,616,Darrius Shepherd,GNB,WR,24.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.90,2019,2.0
26125,617,Jarrett Stidham,NWE,QB,23.0,3.0,0.0,2.0,4.0,14.0,...,0.0,4.0,-2.0,0.0,2.0,0.0,0.0,-1.64,2019,0.0
26126,618,Michael Walker,JAX,WR,23.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,15.0,0.0,-0.50,2019,3.0
